In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población media anual,513170
1,Huelva,Confirmados PDIA,68456
2,Huelva,Confirmados PDIA 14 días,8266
3,Huelva,Tasa PDIA 14 días,"1610,7722587056921"
4,Huelva,Confirmados PDIA 7 días,3262
5,Huelva,Tasa PDIA 7 dias,"635,6567998908744"
6,Huelva,Total Confirmados,68683
7,Huelva,Curados,52629
8,Huelva,Fallecidos,433


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  68456.0


/tmp/ipykernel_20807/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  20064.0


/tmp/ipykernel_20807/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]

/tmp/ipykernel_20807/242626003.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_20807/242626003.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_20807/242626003.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 145 personas en los últimos 7 días 

Un positivo PCR cada 53 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población media anual' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Rename columns
tabla.columns = ['Confirmados PDIA', 'Confirmados PDIA 14 días', 'Confirmados PDIA 7 días', 'Población']

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,68456.0,8266.0,3262.0,513170.0,635.656800,1610.772259,651.0
Huelva-Costa,40017.0,4602.0,1774.0,289548.0,612.679072,1589.373783,436.0
Huelva (capital),20064.0,2726.0,991.0,143837.0,688.974325,1895.200818,248.0
Condado-Campiña,19849.0,1871.0,796.0,156231.0,509.501955,1197.585626,179.0
Cartaya,3017.0,351.0,154.0,20083.0,766.817707,1747.746851,50.0
Moguer,2760.0,264.0,102.0,21867.0,466.456304,1207.298669,34.0
Sierra de Huelva-Andévalo Central,8127.0,1780.0,687.0,67391.0,1019.423959,2641.302251,33.0
Ayamonte,2902.0,251.0,109.0,21104.0,516.489765,1189.347991,30.0
Aljaraque,2719.0,399.0,149.0,21474.0,693.862345,1858.060911,28.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cerro de Andévalo (El),380.0,120.0,40.0,2327.0,1718.951440,5156.854319,2.0
Rosal de la Frontera,205.0,75.0,27.0,1697.0,1591.043017,4419.563936,0.0
Hinojales,30.0,13.0,2.0,330.0,606.060606,3939.393939,0.0
Fuenteheridos,76.0,26.0,8.0,709.0,1128.349788,3667.136812,1.0
Nerva,554.0,188.0,86.0,5169.0,1663.764751,3637.067131,3.0
Cumbres Mayores,283.0,62.0,31.0,1749.0,1772.441395,3544.882790,0.0
Zalamea la Real,299.0,107.0,41.0,3054.0,1342.501637,3503.601834,6.0
Campillo (El),214.0,67.0,21.0,2024.0,1037.549407,3310.276680,1.0
Cortegana,591.0,151.0,93.0,4602.0,2020.860495,3281.182095,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Higuera de la Sierra,119.0,19.0,2.0,1291.0,154.918668,1471.727343,0.0,0.105263
Corteconcepción,72.0,8.0,1.0,536.0,186.567164,1492.537313,0.0,0.125000
Encinasola,199.0,30.0,4.0,1310.0,305.343511,2290.076336,1.0,0.133333
Hinojales,30.0,13.0,2.0,330.0,606.060606,3939.393939,0.0,0.153846
Villalba del Alcor,565.0,26.0,5.0,3366.0,148.544266,772.430184,3.0,0.192308
San Silvestre de Guzmán,83.0,4.0,1.0,614.0,162.866450,651.465798,0.0,0.250000
Nava (La),14.0,4.0,1.0,258.0,387.596899,1550.387597,0.0,0.250000
Almonaster la Real,184.0,37.0,10.0,1788.0,559.284116,2069.351230,1.0,0.270270
San Bartolomé de la Torre,497.0,32.0,9.0,3761.0,239.298059,850.837543,0.0,0.281250
